In [2]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pickle
import os
import json
import numpy as np


Firebase signin

In [3]:

cred = credentials.Certificate("../codeinterface-85b5e-firebase-adminsdk-11q7e-837ba92a03.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [4]:
docs = db.collection('tasks').get()
# save docs in a pickle file called old_tasks.pkl
tasks = {}
for doc in docs:
    doc_dict = doc.to_dict()
    doc_dict['id'] = doc.id 
    tasks[doc.id] = doc_dict

In [5]:

folder_path = './tasks'  # Replace with the actual folder path

# Initialize a list to store all tasks
all_tasks = []

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.json'):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r') as file:
            # Load JSON content
            task_data = json.load(file)
            # Add the task to the list
            all_tasks.append(task_data)

all_tasks = np.array(all_tasks)

In [6]:
all_tasks

array([{'name': 'HumanEval/93', 'function_signature': 'def encode(message):', 'task_description': "Write a function that takes a message, and encodes it in such a way that it swaps the case of all letters, replaces all vowels in the message with the letter that appears 2 places ahead of that vowel in the English alphabet. Assume only letters.\n\nExamples:\n>>> encode('test')\n'TGST'\n>>> encode('This is a message')\n'tHKS KS C MGSSCGG'", 'unit_test': 'assert encode(\'TEST\') == \'tgst\', "This prints if this assert fails 1 (good for debugging!)"\nassert encode(\'Mudasir\') == \'mWDCSKR\', "This prints if this assert fails 2 (good for debugging!)"\nassert encode(\'YES\') == \'ygs\', "This prints if this assert fails 3 (good for debugging!)"\nassert encode(\'This is a message\') == \'tHKS KS C MGSSCGG\', "This prints if this assert fails 2 (also good for debugging!)"\nassert encode("I DoNt KnOw WhAt tO WrItE") == \'k dQnT kNqW wHcT Tq wRkTg\', "This prints if this assert fails 2 (also go

In [11]:
# find task in all_tasks with field name  is Average
def get_task_index(task_name):
    for i in range(len(all_tasks)):
        if all_tasks[i]['name'] == task_name:
            return i
    # raise error if task not found
    raise ValueError('Task not found')
tutorial_task_index = get_task_index('Average')
tasks_chosen = np.array([get_task_index('HumanEval/8'),get_task_index('HumanEval/40'),
                         get_task_index('HumanEval/75'),get_task_index('HumanEval/93')])

In [12]:

# Extracting data for the tutorial task
tutorial_task = all_tasks[tutorial_task_index]

selected_tasks = [all_tasks[i] for i in tasks_chosen]

# Creating the dictionary with the specified structure
fb_taskgpt = {
    'function_signatures': [task['function_signature'] for task in selected_tasks],
    'task_descriptions': [task['task_description'] for task in selected_tasks],
    'tutorial_function_signature': tutorial_task['function_signature'],
    'model': "gpt35",  # As specified, for no model it is None
    'tutorial_task_description': tutorial_task['task_description'],
    'unit_tests': [task['unit_test'] for task in selected_tasks],
    'exp_condition': 'pilot',  # Placeholder, as the actual condition is not specified
    'max_tokens': 64,  # Placeholder, adjust as necessary
    'tutorial_unit_test': tutorial_task['unit_test'],
    'id': "pilot_gpt35"
}

fb_taskllama = {
    'function_signatures': [task['function_signature'] for task in selected_tasks],
    'task_descriptions': [task['task_description'] for task in selected_tasks],
    'tutorial_function_signature': tutorial_task['function_signature'],
    'model': "togethercomputer/CodeLlama-7b",  # As specified, for no model it is None
    'tutorial_task_description': tutorial_task['task_description'],
    'unit_tests': [task['unit_test'] for task in selected_tasks],
    'exp_condition': 'pilot',  
    'max_tokens': 64,  
    'tutorial_unit_test': tutorial_task['unit_test'],
    'id': "pilot_llama7"
}

doc_ref = db.collection(u'tasks').document(u'pilot_gpt35')
doc_ref.set(fb_taskgpt)

doc_ref = db.collection(u'tasks').document(u'pilot_llama7')
doc_ref.set(fb_taskllama)



update_time {
  seconds: 1704070238
  nanos: 379324000
}